In [3]:
#机器之心
#http://mp.weixin.qq.com/s?__biz=MzA3MzI4MjgzMw==&mid=2650730287&idx=1&sn=5697b072a5671ac201e15b43339efab9&chksm=871b2b51b06ca247ac1fdc939e39cfc44cbc2ee25e1b105cf9a582395ddbcdd43cfe18fecd40&mpshare=1&scene=23&srcid=1025xvVK89uXAOZtUNUgwmUH#rd
#=====
#两个模型 一个使用滑动平均 一个没有使用
#都使用L2正则

In [1]:
import numpy as np
import tensorflow as tf
import input_data

In [2]:
def randmize(dataset , labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation , : , :]
    shuffled_labels = labels[permutation]
    
    return shuffled_dataset , shuffled_labels

def one_hot(np_array):
    return (np.arange(10) == np_array[: , None]).astype(np.float32)

def reformat_data(dataset , labels , image_width , image_height , image_depth):
    np_dataset_ = np.array([np.array(image_data).reshape(image_width , image_height , image_depth)])
    np_labels_ = one_hot_encode(np.array(labels , dtype=np.float32))
    np_dataset , np_labels = randmize(np_dataset_ , np_labels_)
    
    return np_dataset , np_labels

def flatten_tf_array(array):
    shape = array.get_shape().as_list()
    return tf.reshape(array , [shape[0] , shape[1]*shape[2]*shape[3]])

def accuracy(predictions , labels):
    return (100.0 * np.sum(np.argmax(predictions , 1) == np.argmax(labels , 1)) / predictions.shape[0])

In [2]:
mnist_folder = 'MNIST_data/'

mnist = input_data.read_data_sets(mnist_folder , one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print(mnist.train.num_examples)
print(mnist.validation.num_examples)
print(mnist.test.num_examples)

55000
5000
10000


In [6]:
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

BATCH_SZIE = 100

regularaztion_rate= 0.001

learning_rate_base = 0.01
learning_rate_decay = 0.99

def inference(input_tensor , avg_class , weights1 , biases1 , weights2, biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor , weights1) + biases1)
        return tf.nn.softmax(tf.matmul(layer1 , weights2) + biases2)
    
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor , avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.nn.softmax(tf.matmul(layer1 , avg_class.average(weights2) + avg_class.average(biases2)))


x = tf.placeholder(tf.float32 , [None , INPUT_NODE])
y_ = tf.placeholder(tf.float32 , [None , OUTPUT_NODE])

#batch norm
x = tf.reshape(x , shape = [-1 , 28,28,1])
mean , variance = tf.nn.moments(x , [0 , 1, 2])
x = tf.nn.batch_normalization(x , mean=mean , variance=variance ,offset= 0, scale=1,variance_epsilon=0.000001 )
x = tf.reshape(x , shape=[-1 , 784])

weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE , LAYER1_NODE] , stddev=0.1))
biases1 = tf.Variable(tf.constant(0.1 , shape = [LAYER1_NODE]))

weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE , OUTPUT_NODE] , stddev=0.1))
biases2 = tf.Variable(tf.constant(0.1 , shape = [OUTPUT_NODE]))

y = inference(x , None , weights1 , biases1 , weights2 , biases2)

cross_entropy_mean = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y+1e-10)))

#增加正则化项 惩戒w1 w2
regularizer = tf.contrib.layers.l2_regularizer(regularaztion_rate)
regularaztion = regularizer(weights1) + regularizer(weights2)
loss = cross_entropy_mean + regularaztion

global_step = tf.Variable(0 , trainable=False)
#==============
#使用滑动平均
MOVING_AVERAGE_DECAY = 0.99
variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
variables_averages_op = variable_averages.apply(tf.trainable_variables())
average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
cross_entropy_mean2 = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(average_y+1e-10)))

loss2 = cross_entropy_mean2 + regularaztion
#==============

#learning rate 递减变化
#初始learning rate
#量
#每多少步更新learning rate
#衰减率
#staircase=True，那就表明每decay_steps次计算学习速率变化，更新原始学习速率，如果是False，那就是每一步都更新学习速率
learning_rate = tf.train.exponential_decay(learning_rate_base,
                                              global_step,
                                              BATCH_SZIE,
                                              learning_rate_decay,
                                              staircase = True)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss , global_step)

with tf.control_dependencies([train_step , variables_averages_op]):
    train_op = tf.no_op(name='train')

sess = tf.Session()
sess.run(tf.global_variables_initializer())

correct_prediction = tf.equal(tf.argmax(y , 1) , tf.argmax(y_ , 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))

validate_feed = {x:mnist.validation.images , y_:mnist.validation.labels}
test_feed = {x:mnist.test.images , y_:mnist.test.labels}


for i in range(1000):
    xs,ys = mnist.train.next_batch(100)
    sess.run(train_op , feed_dict={x:xs , y_:ys})
    #print(sess.run(loss2 , feed_dict={x:xs , y_:ys}))
    if i % 100 == 0:
        print('dd' , sess.run(accuracy , feed_dict=validate_feed))

print('dd' , sess.run(accuracy , feed_dict=test_feed))

dd 0.1918
dd 0.9306
dd 0.9502
dd 0.9584
dd 0.9626
dd 0.964
dd 0.97
dd 0.9726
dd 0.9642
dd 0.9778
dd 0.9723


In [26]:
saver = tf.train.Saver()
saver.save(sess , 'SAVE_TEST/mnist_test/mnist2.ckpt')

'SAVE_TEST/mnist2.ckpt'